# How To Create a Vertex Search AI Data Store with JSON Using Python

This notebook outlines how to create a structured data store in Vertex Search and Conversationl AI. In this example we will generate a JSON data set from items in an image using Gemini-Pro-Vision and then use that data to create the Vertex Search AI data store. After creating the data store, we will perform a query and optionally pass the query and the results to Gemini-Pro to produce the final answer.

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
PROJECT_ID = 'rkiles-demo-host-vpc'
REGION = 'us-central1'
LOCATION = 'global'
#DSNAME = "json-test-datastore" 

Next, let's specify the name of the image file you want to inspect, such as "OJ.png" or "shoe.png"

In [ ]:
image_filename = 'stuff_on_a_shelf.jpg'
#image_filename = 'OJ.png'

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- vertexai - Provides authentication access to the Google API's, such as imagegeneration:predict
- vertexai.preview.generative_models - Interact with new multimodal models
- base64 - Imagen API requests return generated or edited images as base64-encoded strings. This module will help us decode this data to an image file
- json - Python module used to interact with JSON data. Imagen returns results in json format.

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
#from vertexai.generative_models import GenerativeModel, Part

from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1 as vertex_ai

from typing import List
from google.api_core.client_options import ClientOptions

from google.cloud import discoveryengine_v1beta as discoveryengine

import base64
import json

## Instantiate Vertex AI ojbect

To use Gemini Vision Pro on Vertex AI you must provide a text description of what you want to inspect, generate or edit. These descriptions are called prompts, and these prompts are the primary way you communicate with Generative AI. Here, we are specifiying what we want the model to identify using a prompt. Play around with this content and see what kind of details you can extract from an image. More information can be found here https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview

In this example, we will ask Gemini to inspect a picture of an orange juice carton and provide it's results in a json format.

In [ ]:
vqa_prompt = 'Briefly describe each product you see in this picture and provide your response in JSON format. Include the brand, description, size, price, item number, bay and location. If you can not determine the size, mark it as NA. Do not include the json prefix in your response.'
#vqa_prompt = 'Briefly describe each product you see in this picture and provide your response in JSON format. Include the brand, description and size. If you can not determine the size, mark it as NA'

Next we define a function to build the request to be sent to the multimodal model. This will create a base64 encoded string of a local image and uses the from_data function. 

In [ ]:
with open(image_filename, "rb") as f:
    encoded_base_image = base64.b64encode(f.read())
B64_BASE_IMAGE = encoded_base_image.decode('utf-8')

In [ ]:
def generate_text(project_id: str, location: str, b64_image: str, prompt: str) -> str:
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location)
    # Load the model
    multimodal_model = GenerativeModel("gemini-pro-vision")
    # Query the model
    response = multimodal_model.generate_content(
        [
            # Add an example image
            Part.from_data(
                data=base64.b64decode(b64_image), mime_type="image/png"
            ),
            #"what is shown in this image?",
            vqa_prompt,
        ]
    )
    #print(response)
    return response.text

## Send the request and display the response

Call the above generate_text fuction and display the result.

In [ ]:
#qa_response = json.loads(generate_text(PROJECT_ID, LOCATION, B64_BASE_IMAGE, vqa_prompt))
qa_response = generate_text(PROJECT_ID, REGION, B64_BASE_IMAGE, vqa_prompt)
print(qa_response)

Define a function to create a new VAIS Data Store 

In [ ]:
def sample_create_data_store(project_id, location, display_name):
    # Create a client
    client = discoveryengine.DataStoreServiceClient()

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore()
    data_store.display_name = display_name
    data_store.industry_vertical = 'GENERIC'

    request = discoveryengine.CreateDataStoreRequest(
        #parent=f"projects/{project_id}/locations/{location}",
        parent=f"projects/{project_id}/locations/{location}/collections/default_collection",
        data_store=data_store,
        data_store_id = display_name
    )

    # Make the request
    operation = client.create_data_store(request=request)

    print("Waiting for operation to complete...")

    response = operation.result()

    # Handle the response
    print(response)


In [ ]:
#create_and_upload_datastore(PROJECT_ID, LOCATION, DSNAME, qa_response, qa_schema)
sample_create_data_store(PROJECT_ID, LOCATION, DSNAME)

Define a function to create a new document in the Data Store

In [ ]:
def sample_create_document(project_id, location, datastore_name, document_id, json_string):
    # Create a client
    client = discoveryengine.DocumentServiceClient()

    # Initialize request argument(s)
    request = discoveryengine.CreateDocumentRequest(
        parent=f"projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{datastore_name}/branches/0",
        document = discoveryengine.Document(struct_data=json_string),
        document_id=document_id,
    )

    # Make the request
    response = client.create_document(request=request)

    # Handle the response
    print(response)

# [END discoveryengine_v1_generated_DocumentService_CreateDocument_sync]

Create a new document in the data store for each identified item from the image

In [ ]:
qa_json = json.loads(qa_response)

recordnum = 0

for brand in qa_json['products']:
    sample_create_document(PROJECT_ID, LOCATION, DSNAME, 'record'+str(recordnum), brand)
    recordnum = recordnum + 1
    #print(brand)

Define a function to search the data store based on the information from the user prompt

In [ ]:
def search_sample(project_id, location, engine_id, search_query) -> List[discoveryengine.SearchResponse]:

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    
    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)
    
    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"
    
    
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
            ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=""
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="gemini-1.0-pro-001/answer_gen/v1",
            ),
        ),
    )
    
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        #page_size=1,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    #print(response)

    return response

In [ ]:
search_prompt = 'What is the price of Method?'
#search_prompt = 'How many Old English items do I have?'

In [ ]:
#search_result = search_sample(PROJECT_ID, LOCATION, 'test-123_1712092317594', search_prompt)
search_result = search_sample(PROJECT_ID, LOCATION, 'test-123_1712092317594', search_prompt)

print(search_result)

In [ ]:
print(search_result.summary.summary_text)

Notice the summary_text section is blank when using structured data. Summary text is only available for unstructured data stores

In [ ]:
print(search_result.results[0].document.derived_struct_data['snippets'][0]['snippet'])

In order to provide a summary of the returned results we will pass the output from the query to Gemini to provide more details

In [ ]:
def gemini_text(project_id: str, location: str, prompt: str) -> str:
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location)
    # Load the model
    #multimodal_model = GenerativeModel("gemini-pro")
    multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")
    # Query the model
    response = multimodal_model.generate_content(
        [
            #"what is shown in this image?",
            prompt,
        ]
    )
    #print(response)
    return response.text

In [ ]:
gemini_prompt =  '<CONTEXT> ' + str(search_result) + ' </CONTEXT> ' + search_prompt

In [ ]:
final_output = gemini_text(PROJECT_ID, REGION, gemini_prompt)

In [ ]:
print(final_output)

This code will add an unsupporte file type to a data store

In [52]:
def text_create_document(project_id, location, datastore_name, document_id, text_content):
    # Create a client
    client = discoveryengine.DocumentServiceClient()
    
    # Create the Document object
    document = discoveryengine.Document(
        content=discoveryengine.Document.Content(
            mime_type='text/plain',
            raw_bytes=text_content,
            uri='gs://rkiles-test/testing/EX11/extension-packs-templates/EX11/MADTREEX11CustOlpnScanHandlerRoute13.vm',
        ),
        json_data='{"customer_name":"Dollar Tree", "customer_code":"DTRE", "ext_num":"EX11"}'
    )

    # Initialize request argument(s)
    request = discoveryengine.CreateDocumentRequest(
        parent=f"projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{datastore_name}/branches/0",
        document=document,
        document_id=document_id,
    )

    # Make the request
    response = client.create_document(request=request)

    # Handle the response
    print(response)

In [53]:
text_string = 'This is some sample text'
byte_string = bytes(text_string, 'utf-8')

In [54]:
text_create_document(PROJECT_ID, LOCATION, 'manh-test', 'document_id-1234', byte_string)

name: "projects/246771786686/locations/global/collections/default_collection/dataStores/manh-test/branches/0/documents/document_id-1234"
id: "document_id-1234"
schema_id: "default_schema"
json_data: "{\"ext_num\":\"EX11\",\"customer_code\":\"DTRE\",\"customer_name\":\"Dollar Tree\"}"
parent_document_id: "document_id-1234"
content {
  mime_type: "text/plain"
  uri: "gs://rkiles-test/testing/EX11/extension-packs-templates/EX11/MADTREEX11CustOlpnScanHandlerRoute13.vm"
}



In [ ]:
qa_json = json.loads(qa_response)

recordnum = 0

for brand in qa_json['products']:
    sample_create_document(PROJECT_ID, LOCATION, DSNAME, 'record'+str(recordnum), brand)
    recordnum = recordnum + 1
    #print(brand)